In [1]:
from dataset import *
from train import *

In [2]:
import os
import pandas as pd

import collections

import sys

import torch
import torch.optim as optim

In [3]:
# Set parameters
EuroSat_Type = 'ALL'    

In [15]:
args = str(sys.argv)
target_task = args[1]
algorithm = args[2]
algorithm = "bandit"


In [16]:
target_task = 0 #"Moldova"
target_size = 320

In [17]:
from pathlib import Path
output_path = Path("derived_data")
output_path.mkdir(parents = True, exist_ok = True)

In [18]:
if EuroSat_Type == 'RGB':
  data_folder = '/content/sample_data/'
  #root = os.path.join(data_folder, '2750/')
  root = '2750/'
  download_ON = os.path.exists(root)

  if not download_ON:
    # This can be long...
    #os.chdir(data_folder)
    os.system('wget http://madm.dfki.de/files/sentinel/EuroSAT.zip') #Just RGB Bands
    !unzip EuroSAT.zip
    download_ON = True
elif EuroSat_Type == 'ALL':
    root = 'ds/images/remote_sensing/otherDatasets/sentinel_2/tif/'
    download_ON = os.path.exists(root)
    if not download_ON:
      os.system('wget http://madm.dfki.de/files/sentinel/EuroSATallBands.zip') #All bands
      !unzip EuroSATallBands.zip
      download_ON = True

In [19]:
geo_df = pd.read_csv("metadata_clustered10.csv")

In [20]:
data = torchvision.datasets.DatasetFolder(root=root,loader = iloader, transform = None, extensions = 'tif')
labels = [v[1] for (i, v) in enumerate(data)]

In [27]:
input_data = prepare_input_data(geo_df, target_task, group_by = "cluster", 
                                labels = labels,
                               target_size = target_size,
                               val_size = 320, test_size = 320)

In [28]:
for indices in input_data["idx_train"], input_data["idx_val"], input_data["idx_test"], input_data["idx_source"], input_data["source_dict"]["id"]:
    current_labels = [data.samples[i][1] for i in indices]
    print(collections.Counter(current_labels).values())

dict_values([48, 62, 40, 70, 100])
dict_values([39, 83, 81, 65, 52])
dict_values([44, 94, 36, 89, 57])
dict_values([2305, 1742, 2077, 2086, 2647])
dict_values([1741, 2069, 2084, 2647, 2305])


In [22]:
np.random.seed(0)
torch.cuda.manual_seed(0)
random.seed(0)

In [23]:
# prepare data ---

target_val_loader =  torch.utils.data.DataLoader(torch.utils.data.Subset(data, input_data["idx_val"]), 
                                              batch_size = 16, shuffle = True, num_workers = 0)
target_train_loader =  torch.utils.data.DataLoader(torch.utils.data.Subset(data, input_data["idx_train"]), 
                                                  batch_size = 16, shuffle = True, num_workers = 0)
target_test_loader =  torch.utils.data.DataLoader(torch.utils.data.Subset(data, input_data["idx_test"]), 
                                                  batch_size = 16, shuffle = True, num_workers = 0)



# initialize hyperparameters ---

bandit_selects = [None]
alpha = dict.fromkeys(input_data["source_task"], [1])
beta = dict.fromkeys(input_data["source_task"], [1])
pi = dict.fromkeys(input_data["source_task"], [0])


In [24]:
net, bandit_selects, accs, alpha, beta, pi = bandit_selection(data, input_data, 
                                                            n_epochs = 2, n_it = 2,
                                                            algorithm = algorithm, iter_samples = 160,
                                                           output_path = output_path)

/Users/miaoxinran/Desktop/source_selection/experiment_eurosat/dataset.py:97: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-z))


Train (epoch 2/2) [9/10 (90%)]	Loss: 1.007613	Accuracy: 0.5	Learning Rate:0.01
At iteration  1 , source task is  7 , acc is  0.31875


In [25]:
test_performance, y_test, yhat_test = validation(net, target_test_loader)
pd.DataFrame({"test_acc": test_performance,
             "algorithm": algorithm,
             "target_size": target_size,
             "target_task": target_task},
            index = [0]).to_csv(output_path / Path(str(target_task) + "_" + algorithm + "_" + str(target_size) + "_test_acc.csv"))
test_dict = {"y_test": y_test,
               "yhat_test": yhat_test}
pd.DataFrame.from_dict(test_dict).to_csv(output_path / Path(str(target_task) + "_" + algorithm + "_" + str(target_size) + "_test_pred.csv"))